In [ ]:
from time import strftime,localtime
import pandas as pd

In [22]:
df = pd.read_html("https://www.espn.com/nba/stats/player/_/season/2021/seasontype/2/table/offensive/sort/avgPoints/dir/desc")

In [23]:
dfs = df[0].join(df[1])

In [24]:
dfs[['Name','Team']] = dfs['Name'].str.extract('^(.*?)([A-Z]+)$', expand=True)

In [25]:
dfs.head()

,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER,Team
0,1,Stephen Curry,PG,63,34.2,32.0,10.4,21.7,48.2,5.3,...,91.6,5.5,5.8,1.2,0.1,3.4,8,0,26.37,GS
1,2,Bradley Beal,SG,60,35.8,31.3,11.2,23.0,48.5,2.2,...,88.9,4.7,4.4,1.2,0.4,3.1,4,0,22.81,WSH
2,3,Damian Lillard,PG,67,35.8,28.8,9.0,19.9,45.1,4.1,...,92.8,4.2,7.5,0.9,0.3,3.0,16,0,25.65,POR
3,4,Joel Embiid,C,51,31.1,28.5,9.0,17.6,51.3,1.1,...,85.9,10.6,2.8,1.0,1.4,3.1,31,0,30.32,PHI
4,5,Giannis Antetokounmpo,PF,61,33.0,28.1,10.3,18.0,56.9,1.1,...,68.5,11.0,5.9,1.2,1.2,3.4,41,7,29.24,MIL


In [26]:
"https://www.espn.com/nba/stats/player/_/season/2021/seasontype/2/table/offensive/sort/avgPoints/dir/desc"

'https://www.espn.com/nba/stats/player/_/season/2021/seasontype/2/table/offensive/sort/avgPoints/dir/desc'

In [ ]:
def get_team_stats(teams, last_n_years, include_postseason = False):
    this_year = int(strftime("%Y",localtime()))

In [1]:
class Team_standings:
    
    def _init_(self):
        pass
        
    def correct_teams(df):
        teams = df
        first = teams.columns[0]
        row_0 = pd.DataFrame([first],columns=["Team"])
        teams = teams.rename(columns={first:"Team"})
        teams = pd.concat([row_0,teams],ignore_index=True)

        return teams

    def build_table(url):
        dfs = pd.read_html(url)
        stats = dfs[1] 
        teams = correct_teams(dfs[0]) # Needed to correct empty table header on espn site

        return teams.join(stats)

    def gen_key(year, season_type):
        return year+"_"+season_type

    
    def get_team_standings(last_n_years, include_preseason = False):
        REG = 0
        PRE = 1
        standings = {}
        labels =("regular_season", "pre_season")
        this_year = strftime("%Y",localtime())

        root = "https://www.espn.com/nba/standings/_"
        tail = "/group/league"

        this_year_reg_season = root + tail
        standings[gen_key(this_year, labels[REG])] = build_table(this_year_reg_season)

        
        year = int(this_year)
        reg_season_root = root + "/season"
        preseason_root = root + "/seasontype/pre/season"

        for y in range(year,year-last_n_years,-1):
            reg_season = build_url(reg_season_root, str(y-1), tail)
            standings[gen_key(str(y-1),labels[REG])] = build_table(reg_season)
            
            if(include_preseason):
                pre_season = build_url(preseason_root, str(y-1), tail,True)
                standings[gen_key(str(y-1),labels[PRE])] = build_table(pre_season)

        return standings #stats, teams
    
    def build_url(root,year_n, tail,preseason = False):
        if(not preseason):
            return root +  "/" + year_n + tail
        else:
            return root + "/" + year_n + tail
        

In [236]:
standings = get_team_standings(0)

In [237]:
standings

{'2022_r':                          Team   W   L    PCT    GB   HOME   AWAY  DIV   CONF  \
 0     GSGolden State Warriors  29   7  0.806     -   17-3   12-4  7-1   16-4   
 1             PHXPhoenix Suns  29   8  0.784   0.5   16-4   13-4  4-4   19-7   
 2               UTAHUtah Jazz  27  10  0.730   2.5   14-7   13-3  8-0   17-4   
 3            CHIChicago Bulls  25  10  0.714   3.5   13-4   12-6  4-2   16-7   
 4            BKNBrooklyn Nets  23  12  0.657   5.5   10-9   13-3  6-1   17-7   
 5          MILMilwaukee Bucks  25  14  0.641   5.5   13-6   12-8  6-2  17-10   
 6        MEMMemphis Grizzlies  25  14  0.641   5.5   13-8   12-6  4-2   19-9   
 7               MIAMiami Heat  23  15  0.605     7   12-4  11-11  6-1   16-7   
 8       PHIPhiladelphia 76ers  20  16  0.556     9    7-8   13-8  3-6  13-10   
 9      CLECleveland Cavaliers  21  17  0.553     9   11-9   10-8  4-1   14-9   
 10          DENDenver Nuggets  18  17  0.514  10.5    8-6  10-11  2-4  11-10   
 11        DALDall

In [186]:
df

,Team,W,L,PCT,GB,HOME,AWAY,DIV,CONF,PPG,OPP PPG,DIFF,STRK,L10
0,GSGolden State Warriors,29,7,0.806,-,17-3,12-4,7-1,16-4,111.3,101.5,9.8,W2,8-2
1,PHXPhoenix Suns,29,8,0.784,0.5,16-4,13-4,4-4,19-7,112.6,104.9,7.7,W2,7-3
2,UTAHUtah Jazz,27,10,0.730,2.5,14-7,13-3,8-0,17-4,116.0,106.1,9.9,W1,7-3
3,CHIChicago Bulls,25,10,0.714,3.5,13-4,12-6,4-2,16-7,110.7,107.0,3.7,W8,8-2
4,BKNBrooklyn Nets,23,12,0.657,5.5,10-9,13-3,6-1,17-7,110.3,107.4,2.9,L3,6-4
5,MILMilwaukee Bucks,25,14,0.641,5.5,13-6,12-8,6-2,17-10,112.1,107.8,4.3,L1,7-3
6,MEMMemphis Grizzlies,25,14,0.641,5.5,13-8,12-6,4-2,19-9,111.3,108.1,3.2,W6,7-3
7,MIAMiami Heat,23,15,0.605,7,12-4,11-11,6-1,16-7,107.9,104.2,3.7,L2,7-3
8,PHIPhiladelphia 76ers,20,16,0.556,9,7-8,13-8,3-6,13-10,106.8,106.1,0.7,W4,6-4
9,CLECleveland Cavaliers,21,17,0.553,9,11-9,10-8,4-1,14-9,107.7,102.6,5.1,L1,6-4
